In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as msg
from biogeme.expressions import Beta, DefineVariable, log
import math
import numpy as np

In [2]:
data = pd.read_csv("Data.csv",header=0,encoding="UTF-8")

In [3]:
data = data[~data["2_no"].isin(["NaN"])] # 删除第二次未参与回答的回答者
data = data[~data["3_q1"].isin(["NaN"])] # 删除第三次未参与回答的回答者

In [4]:
data.shape[0]

209

In [5]:
data = data[~data["1_group"].isin(["21"])] # 删除身体的な困難のある回答者

In [6]:
data.shape[0]

141

In [7]:
data = data.reset_index(drop=True)

In [8]:
data[106:107]['2_q1']

106    15.0
Name: 2_q1, dtype: float64

In [9]:
data.loc[106,'2_q1'] = 16

In [10]:
data[106:107]['2_q1']

106    16.0
Name: 2_q1, dtype: float64

## Dataset Construction

✓23D(23 district)　1/0 <br />
✓age <br />
✓gender <br />
✓occupation(PTの職業形態と合わせる) <br />
✓vehicle 保有 (dataから作る) <br />
✓commuting transportation mode <br />

In [11]:
def X1_Construction(X,V,Tr):
    # 1_sc1: 性別と年齢
    for i in range(X.shape[0]):
        if X.loc[:,'1_sc1'][i] <= 8:
            #X['gender'][i] = 'male'
            X.loc[i, 'gender'] = 1 #'male'
        if X.loc[:,'1_sc1'][i] > 8 and X.loc[:,'1_sc1'][i] <= 16:
            X.loc[i, 'gender'] = 0 #'female'
        else:
            continue
        
    for i in range(X.shape[0]):
        if X.loc[:,'1_sc1'][i] == 2 or  X.loc[:,'1_sc1'][i] == 10:
            X.loc[i, 'age'] = '20–30'
            X.loc[i, 'a20'] = 1
        if X.loc[:,'1_sc1'][i] == 3 or  X.loc[:,'1_sc1'][i] == 11:
            X.loc[i, 'age'] = '30–40'
            X.loc[i, 'a30'] = 1
        if X.loc[:,'1_sc1'][i] == 4 or  X.loc[:,'1_sc1'][i] == 12:
            X.loc[i, 'age'] = '40–50'
            X.loc[i, 'a40'] = 1
        if X.loc[:,'1_sc1'][i] == 5 or  X.loc[:,'1_sc1'][i] == 13:
            X.loc[i, 'age'] = '50–60'
            X.loc[i, 'a50'] = 1
        if X.loc[:,'1_sc1'][i] == 6 or  X.loc[:,'1_sc1'][i] == 14:
            X.loc[i, 'age'] = '60–65'
            X.loc[i, 'a60'] = 1
        if X.loc[:,'1_sc1'][i] == 7 or  X.loc[:,'1_sc1'][i] == 15:
            X.loc[i, 'age'] = '65–75'
            X.loc[i, 'a65'] = 1
        if X.loc[:,'1_sc1'][i] == 8 or  X.loc[:,'1_sc1'][i] == 16:
            X.loc[i, 'age'] = '75–'
            X.loc[i, 'a75'] = 1    
    
    # 1_sc2: 居住地
    for i in range(X.shape[0]):
        if X.loc[:,'1_sc2'][i] == 13:
            X.loc[i, 'D23'] = 1
        else:
            X.loc[i, 'D23'] = 0    

    # 1_q1, 1_q2: 職業と就業形態
    for i in range(X.shape[0]):
        if X.loc[:, '1_q1'][i] == 4:
            X.loc[i, 'self_employment'] = 1 # 自営業
        if X.loc[:, '1_q2'][i] in range(1,6):
            X.loc[i, 'full_time'] = 1       # 正社員　
        if X.loc[:, '1_q2'][i] in range(7,10):
            X.loc[i, 'temporary'] = 1       # 派遣社員
        if X.loc[:, '1_q1'][i] == 7:
            X.loc[i, 'part_time'] = 1       # パート
        if X.loc[:, '1_q2'][i] == 11:
            X.loc[i, 'officer'] = 1         # 役員
        if X.loc[:, '1_q2'][i] == 15:
            X.loc[i, 'others'] = 1          # その他
        if X.loc[:, '1_q1'][i] == 5:
            X.loc[i, 'student'] = 1         # 学生
        if X.loc[:, '1_q1'][i] == 6:
            X.loc[i, 'household'] = 1       # 専業主婦/主夫
        if X.loc[:, '1_q1'][i] == 8:
            X.loc[i, 'unemployment'] = 1    # 無職
    
    # それぞれの活動を行うためにバスを利用した日数→自家用車の保有
    V = V.replace(1,np.nan)
    V = V.replace('NaN',np.nan)

    VT = pd.DataFrame(V.T.isnull().all())

    for i in range(X.shape[0]):
        if VT.loc[:, 0][i] == False:
            X.loc[i, 'vehicle'] = 1
        else:
            X.loc[i, 'vehicle'] = 0
    
    # 通常時の通勤・通学で利用する交通手段
    Tr = Tr.rename(columns={'1_q3-1':'rail','1_q3-2':'bus',
                       '1_q3-3':'taxi','1_q3-4':'car_self',
                       '1_q3-5':'car_other','1_q3-6':'motorbike',
                       '1_q3-7':'bike','1_q3-8':'others',
                       '1_q3-9':'walking','1_q3-10':'no'})
    
    X = pd.concat([X,Tr],axis=1).fillna(0).drop(columns=['1_sc1','1_sc2','1_q1','1_q2','age'])
    
    return X

In [12]:
def X2_Construction(X,V,Tr):
    # 2_q1: 性別と年齢
    for i in range(X.shape[0]):
        if X.loc[:,'2_q1'][i] <= 8:
            #X['gender'][i] = 'male'
            X.loc[i, 'gender'] = 1 #'male'
        if X.loc[:,'2_q1'][i] > 8 and X.loc[:,'2_q1'][i] <= 16:
            X.loc[i, 'gender'] = 0 #'female'
        else:
            continue
        
    for i in range(X.shape[0]):
        #if X.loc[:,'2_q1'][i] == 2 or  X.loc[:,'2_q1'][i] == 10:
            #X.loc[i, 'age'] = '20–30'
            #X.loc[i, '20–30'] = 1
        if X.loc[:,'2_q1'][i] == 3 or  X.loc[:,'2_q1'][i] == 11:
            X.loc[i, 'age'] = '30–40'
            X.loc[i, 'a30'] = 1
        if X.loc[:,'2_q1'][i] == 4 or  X.loc[:,'2_q1'][i] == 12:
            X.loc[i, 'age'] = '40–50'
            X.loc[i, 'a40'] = 1
        if X.loc[:,'2_q1'][i] == 5 or  X.loc[:,'2_q1'][i] == 13:
            X.loc[i, 'age'] = '50–60'
            X.loc[i, 'a50'] = 1
        if X.loc[:,'2_q1'][i] == 6 or  X.loc[:,'2_q1'][i] == 14:
            X.loc[i, 'age'] = '60–65'
            X.loc[i, 'a60'] = 1
        if X.loc[:,'2_q1'][i] == 7 or  X.loc[:,'2_q1'][i] == 15:
            X.loc[i, 'age'] = '65–75'
            X.loc[i, 'a65'] = 1
        if X.loc[:,'2_q1'][i] == 8 or  X.loc[:,'2_q1'][i] == 16:
            X.loc[i, 'age'] = '75–'
            X.loc[i, 'a75'] = 1    
    
    # 2_q3_2: 居住地
    for i in range(X.shape[0]):
        if X.loc[:,'2_q3_2'][i] <= 23:
            X.loc[i, 'D23'] = 1
        else:
            X.loc[i, 'D23'] = 0    

    # 2_q2_2, 2_q2_3: 職業と就業形態
    for i in range(X.shape[0]):
        if X.loc[:, '2_q2_2'][i] == 4:
            X.loc[i, 'self_employment'] = 1 # 自営業
        if X.loc[:, '2_q2_3'][i] in range(1,6):
            X.loc[i, 'full_time'] = 1       # 正社員　
        if X.loc[:, '2_q2_3'][i] in range(7,10):
            X.loc[i, 'temporary'] = 1       # 派遣社員
        if X.loc[:, '2_q2_2'][i] == 7:
            X.loc[i, 'part_time'] = 1       # パート
        if X.loc[:, '2_q2_3'][i] == 11:
            X.loc[i, 'officer'] = 1         # 役員
        if X.loc[:, '2_q2_3'][i] == 15:
            X.loc[i, 'others'] = 1          # その他
        if X.loc[:, '2_q2_2'][i] == 5:
            X.loc[i, 'student'] = 1         # 学生
        if X.loc[:, '2_q2_2'][i] == 6:
            X.loc[i, 'household'] = 1       # 専業主婦/主夫
        if X.loc[:, '2_q2_2'][i] == 8:
            X.loc[i, 'unemployment'] = 1    # 無職
    
    # それぞれの活動を行うためにバスを利用した日数→自家用車の保有
    V = V.replace(1,np.nan)
    V = V.replace('NaN',np.nan)

    VT = pd.DataFrame(V.T.isnull().all())

    for i in range(X.shape[0]):
        if VT.loc[:, 0][i] == False:
            X.loc[i, 'vehicle'] = 1
        else:
            X.loc[i, 'vehicle'] = 0
    
    # 通常時の通勤・通学で利用する交通手段
    Tr = Tr.rename(columns={'2_q2_4-1':'rail','2_q2_4-2':'bus',
                       '2_q2_4-3':'taxi','2_q2_4-4':'car_self',
                       '2_q2_4-5':'car_other','2_q2_4-6':'motorbike',
                       '2_q2_4-7':'bike','2_q2_4-8':'others',
                       '2_q2_4-9':'walking','2_q2_4-10':'no'})
    
    X = pd.concat([X,Tr],axis=1).fillna(0).drop(columns=['2_q1','2_q2_2','2_q2_3','2_q3_2','age'])
    
    return X

In [13]:
def X3_Construction(X,V,Tr):
    # 3_q1: 性別と年齢
    for i in range(X.shape[0]):
        if X.loc[:,'3_q1'][i] <= 8:
            #X['gender'][i] = 'male'
            X.loc[i, 'gender'] = 1 #'male'
        if X.loc[:,'3_q1'][i] > 8 and X.loc[:,'3_q1'][i] <= 16:
            X.loc[i, 'gender'] = 0 #'female'
        else:
            continue
        
    for i in range(X.shape[0]):
        #if X.loc[:,'3_q1'][i] == 2 or  X.loc[:,'3_q1'][i] == 10:
            #X.loc[i, 'age'] = '20–30'
            #X.loc[i, '20–30'] = 1
        if X.loc[:,'3_q1'][i] == 3 or  X.loc[:,'3_q1'][i] == 11:
            X.loc[i, 'age'] = '30–40'
            X.loc[i, 'a30'] = 1
        if X.loc[:,'3_q1'][i] == 4 or  X.loc[:,'3_q1'][i] == 12:
            X.loc[i, 'age'] = '40–50'
            X.loc[i, 'a40'] = 1
        if X.loc[:,'3_q1'][i] == 5 or  X.loc[:,'3_q1'][i] == 13:
            X.loc[i, 'age'] = '50–60'
            X.loc[i, 'a50'] = 1
        if X.loc[:,'3_q1'][i] == 6 or  X.loc[:,'3_q1'][i] == 14:
            X.loc[i, 'age'] = '60–65'
            X.loc[i, 'a60'] = 1
        if X.loc[:,'3_q1'][i] == 7 or  X.loc[:,'3_q1'][i] == 15:
            X.loc[i, 'age'] = '65–75'
            X.loc[i, 'a65'] = 1
        if X.loc[:,'3_q1'][i] == 8 or  X.loc[:,'3_q1'][i] == 16:
            X.loc[i, 'age'] = '75–'
            X.loc[i, 'a75'] = 1    
    
    # 3_q3_2: 居住地
    for i in range(X.shape[0]):
        if X.loc[:,'3_q3_2'][i] <= 23:
            X.loc[i, 'D23'] = 1
        else:
            X.loc[i, 'D23'] = 0    

    # 3_q2_2, 3_q2_3: 職業と就業形態
    for i in range(X.shape[0]):
        if X.loc[:, '3_q2_2'][i] == 4:
            X.loc[i, 'self_employment'] = 1 # 自営業
        if X.loc[:, '3_q2_3'][i] in range(1,6):
            X.loc[i, 'full_time'] = 1       # 正社員　
        if X.loc[:, '3_q2_3'][i] in range(7,10):
            X.loc[i, 'temporary'] = 1       # 派遣社員
        if X.loc[:, '3_q2_2'][i] == 7:
            X.loc[i, 'part_time'] = 1       # パート
        if X.loc[:, '3_q2_3'][i] == 11:
            X.loc[i, 'officer'] = 1         # 役員
        if X.loc[:, '3_q2_3'][i] == 15:
            X.loc[i, 'others'] = 1          # その他
        if X.loc[:, '3_q2_2'][i] == 5:
            X.loc[i, 'student'] = 1         # 学生
        if X.loc[:, '3_q2_2'][i] == 6:
            X.loc[i, 'household'] = 1       # 専業主婦/主夫
        if X.loc[:, '3_q2_2'][i] == 8:
            X.loc[i, 'unemployment'] = 1    # 無職
    
    # それぞれの活動を行うためにバスを利用した日数→自家用車の保有
    V = V.replace(1,np.nan)
    V = V.replace('NaN',np.nan)

    VT = pd.DataFrame(V.T.isnull().all())

    for i in range(X.shape[0]):
        if VT.loc[:, 0][i] == False:
            X.loc[i, 'vehicle'] = 1
        else:
            X.loc[i, 'vehicle'] = 0
    
    # 通常時の通勤・通学で利用する交通手段
    Tr = Tr.rename(columns={'3_q2_4-1':'rail','3_q2_4-2':'bus',
                       '3_q2_4-3':'taxi','3_q2_4-4':'car_self',
                       '3_q2_4-5':'car_other','3_q2_4-6':'motorbike',
                       '3_q2_4-7':'bike','3_q2_4-8':'others',
                       '3_q2_4-9':'walking','3_q2_4-10':'no'})
    
    X = pd.concat([X,Tr],axis=1).fillna(0).drop(columns=['3_q1','3_q2_2','3_q2_3','3_q3_2','age'])
    
    return X

In [14]:
X1 = data.loc[:,['1_sc1','1_sc2','1_q1','1_q2']]

V = pd.concat([data.loc[:,'1_q6_7_1':'1_q6_7_12'],
               data.loc[:,'2_q4_7_1':'2_q4_7_12'],
               data.loc[:,'3_q4_7_1':'3_q4_7_12']],axis=1) 

Tr1 = data.loc[:,'1_q3-1':'1_q3-10']

X1 = X1_Construction(X1,V,Tr1)

In [15]:
X2 = data.loc[:,['2_q1','2_q2_2','2_q2_3','2_q3_2']]

V = pd.concat([data.loc[:,'1_q6_7_1':'1_q6_7_12'],
               data.loc[:,'2_q4_7_1':'2_q4_7_12'],
               data.loc[:,'3_q4_7_1':'3_q4_7_12']],axis=1) 

Tr2 = data.loc[:,'2_q2_4-1':'2_q2_4-10'] 

X2 = X2_Construction(X2,V,Tr2)

In [16]:
X3 = data.loc[:,['3_q1','3_q2_2','3_q2_3','3_q3_2']]

V = pd.concat([data.loc[:,'1_q6_7_1':'1_q6_7_12'],
               data.loc[:,'2_q4_7_1':'2_q4_7_12'],
               data.loc[:,'3_q4_7_1':'3_q4_7_12']],axis=1) 

Tr3 = data.loc[:,'3_q2_4-1':'3_q2_4-10'] 

X3 = X3_Construction(X3,V,Tr3)

In [17]:
X = pd.concat([X1,X1,X2,X3]).fillna(0)

In [18]:
Y1 = data.loc[:,'1_q8_2_1':'1_q8_2_12'].fillna(0)
Y2 = data.loc[:,'1_q6_2_1':'1_q6_2_12'].fillna(0)
Y3 = data.loc[:,'2_q4_2_1':'2_q4_2_12'].fillna(0)
Y4 = data.loc[:,'3_q4_2_1':'3_q4_2_12'].fillna(0)

Y1.columns = ['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12']
Y2.columns = ['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12']
Y3.columns = ['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12']
Y4.columns = ['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12']

In [19]:
Y = pd.concat([Y1,Y2,Y3,Y4])

In [20]:
Database = pd.concat([X,Y],axis=1)
Database

,gender,a20,a30,a65,a40,a60,a50,a75,D23,temporary,...,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,3.0,5.0,5.0,1.0,0.0,2.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
138,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
139,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
Database.columns.values.tolist()

['gender',
 'a20',
 'a30',
 'a65',
 'a40',
 'a60',
 'a50',
 'a75',
 'D23',
 'temporary',
 'full_time',
 'household',
 'self_employment',
 'unemployment',
 'part_time',
 'officer',
 'student',
 'vehicle',
 'rail',
 'bus',
 'taxi',
 'car_self',
 'car_other',
 'motorbike',
 'bike',
 'others',
 'walking',
 'no',
 'a1',
 'a2',
 'a3',
 'a4',
 'a5',
 'a6',
 'a7',
 'a8',
 'a9',
 'a10',
 'a11',
 'a12']

## Model Estimation 

In [22]:
database = db.Database('Database', Database)

In [23]:
globals().update(database.variables)

✓23D(23 district)　1/0 <br />
✓age <br />
✓gender <br />
✓occupation(PTの職業業態と合わせる) <br />
✓vehicle 保有 (dataから作る) <br />
✓commuting transportation mode <br />

In [24]:
ASC_10 = Beta('ASC_10', 0, None, None, 0)
ASC_11 = Beta('ASC_11', 0, None, None, 0)
ASC_12 = Beta('ASC_12', 0, None, None, 0)
ASC_13 = Beta('ASC_13', 0, None, None, 0)
ASC_14 = Beta('ASC_14', 0, None, None, 0)
ASC_15 = Beta('ASC_15', 0, None, None, 0)
ASC_16 = Beta('ASC_16', 0, None, None, 0)
ASC_17 = Beta('ASC_17', 0, None, None, 1)

B_gen1 = Beta('B_gen1', 0, None, None, 0)

B_23D1 = Beta('B_23D1', 0, None, None, 0)

B_veh1 = Beta('B_veh1', 0, None, None, 0)

#B_2030 = Beta('B_2030', 0, None, None, 0)
B_30401 = Beta('B_30401', 0, None, None, 0)
B_40501 = Beta('B_40501', 0, None, None, 0)
B_50601 = Beta('B_50601', 0, None, None, 0)
B_60651 = Beta('B_60651', 0, None, None, 0)
B_65751 = Beta('B_65751', 0, None, None, 0)
B_751 = Beta('B_751', 0, None, None, 0)

#B_sel = Beta('B_sel', 0, None, None, 0)
B_tem1 = Beta('B_tem1', 0, None, None, 0)
B_ful1 = Beta('B_ful1', 0, None, None, 0)
B_hou1 = Beta('B_hou1', 0, None, None, 0)
B_une1 = Beta('B_une1', 0, None, None, 0)
B_par1 = Beta('B_par1', 0, None, None, 0)
B_off1 = Beta('B_off1', 0, None, None, 0)
B_stu1 = Beta('B_stu1', 0, None, None, 0)

#B_rai = Beta('B_rai', 0, None, None, 0)
B_bus1 = Beta('B_bus1', 0, None, None, 0)
B_tax1 = Beta('B_tax1', 0, None, None, 0)
B_cars1 = Beta('B_cars1', 0, None, None, 0)
B_caro1 = Beta('B_caro1', 0, None, None, 0)
B_mot1 = Beta('B_mot1', 0, None, None, 0)
B_bik1 = Beta('B_bik1', 0, None, None, 0)
B_oth1 = Beta('B_oth1', 0, None, None, 0)
B_wak1 = Beta('B_wak1', 0, None, None, 0)
B_no1 = Beta('B_no1', 0, None, None, 0)

In [25]:
ASC_20 = Beta('ASC_20', 0, None, None, 0)
ASC_21 = Beta('ASC_21', 0, None, None, 0)
ASC_22 = Beta('ASC_22', 0, None, None, 0)
ASC_23 = Beta('ASC_23', 0, None, None, 0)
ASC_24 = Beta('ASC_24', 0, None, None, 0)
ASC_25 = Beta('ASC_25', 0, None, None, 0)
ASC_26 = Beta('ASC_26', 0, None, None, 0)
ASC_27 = Beta('ASC_27', 0, None, None, 1)

B_gen2 = Beta('B_gen2', 0, None, None, 0)

B_23D2 = Beta('B_23D2', 0, None, None, 0)

B_veh2 = Beta('B_veh2', 0, None, None, 0)

#B_2030 = Beta('B_2030', 0, None, None, 0)
B_30402 = Beta('B_30402', 0, None, None, 0)
B_40502 = Beta('B_40502', 0, None, None, 0)
B_50602 = Beta('B_50602', 0, None, None, 0)
B_60652 = Beta('B_60652', 0, None, None, 0)
B_65752 = Beta('B_65752', 0, None, None, 0)
B_752 = Beta('B_752', 0, None, None, 0)

#B_sel = Beta('B_sel', 0, None, None, 0)
B_tem2 = Beta('B_tem2', 0, None, None, 0)
B_ful2 = Beta('B_ful2', 0, None, None, 0)
B_hou2 = Beta('B_hou2', 0, None, None, 0)
B_une2 = Beta('B_une2', 0, None, None, 0)
B_par2 = Beta('B_par2', 0, None, None, 0)
B_off2 = Beta('B_off2', 0, None, None, 0)
B_stu2 = Beta('B_stu2', 0, None, None, 0)

#B_rai = Beta('B_rai', 0, None, None, 0)
B_bus2 = Beta('B_bus2', 0, None, None, 0)
B_tax2 = Beta('B_tax2', 0, None, None, 0)
B_cars2 = Beta('B_cars2', 0, None, None, 0)
B_caro2 = Beta('B_caro2', 0, None, None, 0)
B_mot2 = Beta('B_mot2', 0, None, None, 0)
B_bik2 = Beta('B_bik2', 0, None, None, 0)
B_oth2 = Beta('B_oth2', 0, None, None, 0)
B_wak2 = Beta('B_wak2', 0, None, None, 0)
B_no2 = Beta('B_no2', 0, None, None, 0)

In [26]:
PROB_CLASS1 = Beta('PROB_CLASS1', 0.5, 0, 1, 0)
PROB_CLASS2 = 1 - PROB_CLASS1

In [27]:
V10 = ASC_10 + \
       B_gen1*gender + B_23D1*D23 + B_veh1*vehicle + \
       B_30401*a30 + B_40501*a40 + B_50601*a50 + B_60651*a60 + B_65751*a65 + B_751*a75 + \
       B_tem1*temporary + B_ful1*full_time + B_hou1*household + B_une1*unemployment + B_par1*part_time + B_off1*officer + B_stu1*student

V11 = ASC_11 + \
       B_gen1*gender + B_23D1*D23 + B_veh1*vehicle + \
       B_30401*a30 + B_40501*a40 + B_50601*a50 + B_60651*a60 + B_65751*a65 + B_751*a75 + \
       B_tem1*temporary + B_ful1*full_time + B_hou1*household + B_une1*unemployment + B_par1*part_time + B_off1*officer + B_stu1*student

V12 = ASC_12 + \
       B_gen1*gender + B_23D1*D23 + B_veh1*vehicle + \
       B_30401*a30 + B_40501*a40 + B_50601*a50 + B_60651*a60 + B_65751*a65 + B_751*a75 + \
       B_tem1*temporary + B_ful1*full_time + B_hou1*household + B_une1*unemployment + B_par1*part_time + B_off1*officer + B_stu1*student

V13 = ASC_13 + \
       B_gen1*gender + B_23D1*D23 + B_veh1*vehicle + \
       B_30401*a30 + B_40501*a40 + B_50601*a50 + B_60651*a60 + B_65751*a65 + B_751*a75 + \
       B_tem1*temporary + B_ful1*full_time + B_hou1*household + B_une1*unemployment + B_par1*part_time + B_off1*officer + B_stu1*student

V14 = ASC_14 + \
       B_gen1*gender + B_23D1*D23 + B_veh1*vehicle + \
       B_30401*a30 + B_40501*a40 + B_50601*a50 + B_60651*a60 + B_65751*a65 + B_751*a75 + \
       B_tem1*temporary + B_ful1*full_time + B_hou1*household + B_une1*unemployment + B_par1*part_time + B_off1*officer + B_stu1*student

V15 = ASC_15 + \
       B_gen1*gender + B_23D1*D23 + B_veh1*vehicle + \
       B_30401*a30 + B_40501*a40 + B_50601*a50 + B_60651*a60 + B_65751*a65 + B_751*a75 + \
       B_tem1*temporary + B_ful1*full_time + B_hou1*household + B_une1*unemployment + B_par1*part_time + B_off1*officer + B_stu1*student

V16 = ASC_16 + \
       B_gen1*gender + B_23D1*D23 + B_veh1*vehicle + \
       B_30401*a30 + B_40501*a40 + B_50601*a50 + B_60651*a60 + B_65751*a65 + B_751*a75 + \
       B_tem1*temporary + B_ful1*full_time + B_hou1*household + B_une1*unemployment + B_par1*part_time + B_off1*officer + B_stu1*student

V17 = ASC_17 + \
       B_gen1*gender + B_23D1*D23 + B_veh1*vehicle + \
       B_30401*a30 + B_40501*a40 + B_50601*a50 + B_60651*a60 + B_65751*a65 + B_751*a75 + \
       B_tem1*temporary + B_ful1*full_time + B_hou1*household + B_une1*unemployment + B_par1*part_time + B_off1*officer + B_stu1*student

In [28]:
# Associate utility functions with the numbering of alternatives
V1 = {0: V11,
      1: V12,
      2: V12,
      3: V13,
      4: V14,
      5: V15,
      6: V16,
      7: V17}

In [29]:
V20 = ASC_20 + \
       B_gen2*gender + B_23D2*D23 + B_veh2*vehicle + \
       B_30402*a30 + B_40502*a40 + B_50602*a50 + B_60652*a60 + B_65752*a65 + B_752*a75 + \
       B_tem2*temporary + B_ful2*full_time + B_hou2*household + B_une2*unemployment + B_par2*part_time + B_off2*officer + B_stu2*student
V21 = ASC_21 + \
       B_gen2*gender + B_23D2*D23 + B_veh2*vehicle + \
       B_30402*a30 + B_40502*a40 + B_50602*a50 + B_60652*a60 + B_65752*a65 + B_752*a75 + \
       B_tem2*temporary + B_ful2*full_time + B_hou2*household + B_une2*unemployment + B_par2*part_time + B_off2*officer + B_stu2*student

V22 = ASC_22 + \
       B_gen2*gender + B_23D2*D23 + B_veh2*vehicle + \
       B_30402*a30 + B_40502*a40 + B_50602*a50 + B_60652*a60 + B_65752*a65 + B_752*a75 + \
       B_tem2*temporary + B_ful2*full_time + B_hou2*household + B_une2*unemployment + B_par2*part_time + B_off2*officer + B_stu2*student
V23 = ASC_23 + \
       B_gen2*gender + B_23D2*D23 + B_veh2*vehicle + \
       B_30402*a30 + B_40502*a40 + B_50602*a50 + B_60652*a60 + B_65752*a65 + B_752*a75 + \
       B_tem2*temporary + B_ful2*full_time + B_hou2*household + B_une2*unemployment + B_par2*part_time + B_off2*officer + B_stu2*student

V24 = ASC_24 + \
       B_gen2*gender + B_23D2*D23 + B_veh2*vehicle + \
       B_30402*a30 + B_40502*a40 + B_50602*a50 + B_60652*a60 + B_65752*a65 + B_752*a75 + \
       B_tem2*temporary + B_ful2*full_time + B_hou2*household + B_une2*unemployment + B_par2*part_time + B_off2*officer + B_stu2*student
V25 = ASC_25 + \
       B_gen2*gender + B_23D2*D23 + B_veh2*vehicle + \
       B_30402*a30 + B_40502*a40 + B_50602*a50 + B_60652*a60 + B_65752*a65 + B_752*a75 + \
       B_tem2*temporary + B_ful2*full_time + B_hou2*household + B_une2*unemployment + B_par2*part_time + B_off2*officer + B_stu2*student

V26 = ASC_26 + \
       B_gen2*gender + B_23D2*D23 + B_veh2*vehicle + \
       B_30402*a30 + B_40502*a40 + B_50602*a50 + B_60652*a60 + B_65752*a65 + B_752*a75 + \
       B_tem2*temporary + B_ful2*full_time + B_hou2*household + B_une2*unemployment + B_par2*part_time + B_off2*officer + B_stu2*student

V27 = ASC_27 + \
       B_gen2*gender + B_23D2*D23 + B_veh2*vehicle + \
       B_30402*a30 + B_40502*a40 + B_50602*a50 + B_60652*a60 + B_65752*a65 + B_752*a75 + \
       B_tem2*temporary + B_ful2*full_time + B_hou2*household + B_une2*unemployment + B_par2*part_time + B_off2*officer + B_stu2*student 

In [30]:
# Associate utility functions with the numbering of alternatives
V2 = {0: V20,
      1: V21,
      2: V22,
      3: V23,
      4: V24,
      5: V25,
      6: V26,
      7: V27}

In [31]:
av = {0: 1,
      1: 1,
      2: 1,
      3: 1,
      4: 1,
      5: 1,
      6: 1,
      7: 1}

In [32]:
# The choice model is a discrete mixture of logit, with availability conditions
prob1a1 = models.logit(V1, av, a1)
prob1a2 = models.logit(V1, av, a2)
prob1a3 = models.logit(V1, av, a3)
prob1a4 = models.logit(V1, av, a4)
prob1a5 = models.logit(V1, av, a5)
prob1a6 = models.logit(V1, av, a6)
prob1a7 = models.logit(V1, av, a7)
prob1a8 = models.logit(V1, av, a8)
prob1a9 = models.logit(V1, av, a9)
prob1a10 = models.logit(V1, av, a10)
prob1a11 = models.logit(V1, av, a11)
prob1a12 = models.logit(V1, av, a12)

prob2a1 = models.logit(V2, av, a1)
prob2a2 = models.logit(V2, av, a2)
prob2a3 = models.logit(V2, av, a3)
prob2a4 = models.logit(V2, av, a4)
prob2a5 = models.logit(V2, av, a5)
prob2a6 = models.logit(V2, av, a6)
prob2a7 = models.logit(V2, av, a7)
prob2a8 = models.logit(V2, av, a8)
prob2a9 = models.logit(V2, av, a9)
prob2a10 = models.logit(V2, av, a10)
prob2a11 = models.logit(V2, av, a11)
prob2a12 = models.logit(V2, av, a12)

prob = PROB_CLASS1*prob1a1*prob1a2*prob1a3*prob1a4*prob1a5*prob1a6*prob1a7*prob1a8*prob1a9*prob1a10*prob1a11*prob1a12 + \
       PROB_CLASS2*prob2a1*prob2a2*prob2a3*prob2a4*prob2a5*prob2a6*prob2a7*prob2a8*prob2a9*prob2a10*prob2a11*prob2a12

logprob = log(prob)

In [33]:
# Define level of verbosity
logger = msg.bioMessage()
logger.setSilent()
#logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = '07discreteMixture'

# Estimate the parameters
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)

                Value        Std err     t-test   p-value   Rob. Std err  \
ASC_11       7.377014   2.312004e+01   0.319074  0.749670   1.000138e+00   
ASC_12       1.118644   1.466965e+03   0.000763  0.999392   1.099627e-01   
ASC_13      -6.060013   4.783021e+02  -0.012670  0.989891   2.768373e-03   
ASC_14       8.223805   2.311619e+01   0.355759  0.722021   1.000117e+00   
ASC_15       6.970935   2.312381e+01   0.301461  0.763063   1.000289e+00   
ASC_16      -5.829109   4.262491e+02  -0.013675  0.989089   2.764447e-03   
ASC_20       4.754594   1.497153e-01  31.757576  0.000000   1.667678e-01   
ASC_21       2.566705   1.546925e-01  16.592306  0.000000   1.731213e-01   
ASC_22       1.913701   1.596933e-01  11.983605  0.000000   1.768479e-01   
ASC_23       1.674029   1.624498e-01  10.304901  0.000000   1.767293e-01   
ASC_24       0.788503   1.797897e-01   4.385693  0.000012   1.998334e-01   
ASC_25       1.577882   1.637389e-01   9.636572  0.000000   1.791418e-01   
ASC_26      